<a href="https://colab.research.google.com/github/iaZe/Ideias-e-Aprendizados/blob/main/Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=054c7cd1055f3009a11865bb108e7c1b06bb6efc7a0fcb1822c7cf4d247cbb7c
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [90]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, asc, desc

spark = SparkSession.builder.appName("Spotyfinho").getOrCreate()

data_path = "/content/spotify.csv"
dataframe = spark.read.csv(data_path, header = True, inferSchema = True)

dataframe.show()

+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|         0|5SuOikwiRyPMVoIQD...|         Gen Hoshino|              Comedy|              Comedy|        73|     230666|   False|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|         1.01E-6|   0.358|

In [142]:
dataframe_view = dataframe.select("track_name", "artists", "album_name", "duration_ms", "popularity", "track_genre")
dataframe_view = dataframe.filter(col("popularity").cast("double").isNotNull())
dataframe_view.show()

+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+----------+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|         0|5SuOikwiRyPMVoIQD...|         Gen Hoshino|              Comedy|              Comedy|        73|     230666|   False|       0.676| 0.461|  1|  -6.746|   0|      0.143|      0.0322|         1.01E-6|   0.358|

In [128]:
dataframe_view.createOrReplaceTempView("view")

Mostrar o TOP 10 do spotify

In [129]:
most_popular_music = spark.sql("SELECT track_name, artists, popularity FROM view ORDER BY popularity DESC LIMIT 10")
most_popular_music.show()

+--------------------+--------------------+----------+
|          track_name|             artists|popularity|
+--------------------+--------------------+----------+
|Quevedo: Bzrp Mus...|    Bizarrap;Quevedo|        99|
|     I'm Good (Blue)|David Guetta;Bebe...|        98|
|     I'm Good (Blue)|David Guetta;Bebe...|        98|
|          La Bachata|       Manuel Turizo|        98|
|     I'm Good (Blue)|David Guetta;Bebe...|        98|
|          La Bachata|       Manuel Turizo|        98|
|          La Bachata|       Manuel Turizo|        98|
|          La Bachata|       Manuel Turizo|        98|
|     Me Porto Bonito|Bad Bunny;Chencho...|        97|
|     Me Porto Bonito|Bad Bunny;Chencho...|        97|
+--------------------+--------------------+----------+



Mostrar a música mais famosa do spotify

In [152]:
most_popular_album = spark.sql("SELECT * FROM view WHERE album_name = (SELECT album_name FROM view ORDER BY popularity DESC LIMIT 1)")
most_popular_album.show()

+----------+--------------------+----------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|Unnamed: 0|            track_id|         artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+----------+--------------------+----------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|     51664|2tTmW7RDtMQtBk7m2...|Bizarrap;Quevedo|Quevedo: Bzrp Mus...|Quevedo: Bzrp Mus...|        99|     198937|   False|       0.621| 0.782|  2|  -5.548|   1|      0.044|      0.0125|           0.033|    0.23|   0.55|128.033|

Mostrar os albuns de um(a) cantor(a)

In [131]:
singer = "Billie Eilish"
filter_singer = dataframe_view.select("artists", "album_name").filter(dataframe_view.artists == singer)
filter_singer.show()

+-------------+--------------------+
|      artists|          album_name|
+-------------+--------------------+
|Billie Eilish|WHEN WE ALL FALL ...|
|Billie Eilish|   Halloween & Chill|
|Billie Eilish|      pov: it's 2021|
|Billie Eilish|   Отново на училище|
|Billie Eilish|Halloween Songs |...|
|Billie Eilish|   Halloween & Chill|
|Billie Eilish|Twisted Halloween...|
|Billie Eilish|   Halloween & Chill|
|Billie Eilish|Halloween Party 2022|
|Billie Eilish|Halloween & Alter...|
|Billie Eilish|         Girls Night|
|Billie Eilish|Halloween Party 2022|
|Billie Eilish|      Halloween 2022|
|Billie Eilish|     Pop & Halloween|
|Billie Eilish|      Herbststimmung|
|Billie Eilish|Halloween & Alter...|
|Billie Eilish|Beginner Ride (Sw...|
|Billie Eilish|      Halloween 2022|
|Billie Eilish|Halloween Party 2022|
|Billie Eilish|Halloween Party 2022|
+-------------+--------------------+
only showing top 20 rows



Mostrar a média de tempo de música dos artistas mais ouvidos

In [151]:
average_time_popular_songs = spark.sql("SELECT artists, AVG(duration_ms) FROM view WHERE popularity >= 90 GROUP BY artists LIMIT 10")
average_time_popular_songs.show()

+--------------------+------------------+
|             artists|  avg(duration_ms)|
+--------------------+------------------+
|           Lil Nas X|          210575.0|
|James Hype;Miggy ...|          186661.0|
|       Manuel Turizo|          162637.0|
|   The Neighbourhood|          240400.0|
|    Bad Bunny;Jhayco|          237894.0|
|           Tom Odell|          244360.0|
|Charlie Puth;Jung...|          154486.0|
|           Bad Bunny|222014.73333333334|
|             Seafret|          229173.0|
|         Nicki Minaj|          170977.0|
+--------------------+------------------+



Mostrar os ritmos mais populares

In [165]:
dataframe_genre = dataframe_view.select("track_genre").distinct().orderBy(col("popularity").asc()).show()
dataframe_genre

+--------------+
|   track_genre|
+--------------+
|         latin|
|         salsa|
|        pagode|
|         party|
|        reggae|
|     grindcore|
|       iranian|
|         tango|
|       romance|
|   heavy-metal|
|          kids|
|         samba|
|      cantopop|
|     breakbeat|
|        gospel|
|         forro|
|          goth|
|   black-metal|
|detroit-techno|
|         study|
+--------------+
only showing top 20 rows

